In [ ]:
import numpy as np
pi = np.array([0.6, 0.4])
A  = np.array([[0.7, 0.3],
               [0.4, 0.6]])
B  = np.array([[0.5, 0.4, 0.1],
               [0.1, 0.3, 0.6]])

obs = [0, 1, 2, 1, 0]

In [ ]:
def forward(A,B,obs,pi):
  T=len(obs)
  N=len(pi)
  alpha=np.zeros((T,N))
  alpha[0]=pi*B[:,obs[0]]
  for t in range(1,T):
    for j in range(N):
      alpha[t,j]=np.sum(alpha[t-1]*A[:,j])*B[j,obs[t]]
  return np.sum(alpha[-1]),alpha

In [ ]:
def viterbi(A,B,obs,pi):
  T=len(obs)
  N=len(pi)
  delta=np.zeros((T,N))
  psi=np.zeros((T,N),dtype=int)

  delta[0]=pi*B[:,obs[0]]

  for t in range(1,T):
    for j in range(N):
      seq_probs=delta[t-1]*A[:,j]
      psi[t,j]=np.argmax(seq_probs)
      delta[t,j]=np.max(seq_probs)*B[j,obs[t]]

  last_state=np.argmax(delta[-1])
  path=[last_state]
  for t in range(T-1,0,-1):
      path.insert(0,psi[t,path[0]])
  return path,delta


In [ ]:
forward_prob,alpha_table=forward(A,B,obs,pi)
path,delta_table=viterbi(A,B,obs,pi)
print(forward_prob)
print(path)

0.0040745792
[np.int64(0), np.int64(0), np.int64(0), np.int64(0), np.int64(0)]


# ctc

In [ ]:
def logsumex_vec(v):
  m=np.max(v)
  return m+math.log(np.sum(np.exp(v-m)))


def expand_with_blank(labels,blank=0):
  ex=[blank]
  for l in labels:
    ex.append(l)
    ex.append(blank)
  return ex


def ctc_forward(probs,labels,blank=0):
  T,c=probs.shape
  ext=expand_with_blank(labels,blank)
  s=len(ext)
  logp=np.log(probs+1e-12)
  alpha=np.full((T,s),-np.inf)
  alpha[0,0]=logp[0,ext[0]]
  if s>1:
    alpha[0,1]=logp[0,ext[1]]
  for t in range(1,T):
    for s in range(s):
      terms=[alpha[t-1,s]]
      if s>0:
        terms.append(alpha[t-1,s-1])
      if s>1 and ext[s] != blank and ext[s] != ext[s-2]:
        terms.append(alpha[t-1,s-2])
      alpha[t,s]=logsumexp_vec(np.array(terms))+logp[t,ext[s]]
  if s==1:
    return alpha[T-1,0]
  return logsumexp_vec(np.array([alpha[t-1,s-1],alpha[t-1,s-2]]))


In [ ]:
import numpy as np
import math

def logsumexp_vec(v):
  m=np.max(v)
  return m + math.log(np.sum(np.exp(v-m)))

def expand_with_blank(labels,blank=0):
  ex=[blank]
  for l in labels:
    ex.append(l)
    ex.append(blank)
  return ex

def ctc_forward(probs,labels,blank=0):
  T,c=probs.shape
  ext=expand_with_blank(labels,blank)
  s=len(ext)
  logp=np.log(probs+1e-12)
  alpha=np.full((T,s),-np.inf)
  alpha[0,0]=logp[0,ext[0]]
  if s>1:
    alpha[0,1]=logp[0,ext[1]]
  for t in range(1,T):
    for s in range(s):
      terms=[alpha[t-1,s]]
      if s>0:
        terms.append(alpha[t-1,s-1])
      if s>1 and ext[s]!=blank and ext[s]!=ext[s-2]:
        terms.append(alpha[t-1,s-2])
      alpha[t,s]=logsumexp_vec(np.array(terms))+logp[t,ext[t]]
  if s==1:
    return [T-1,0]
  return logsumexp_vec(np.array([alpha[t-1,s-1],alpha[t-1,s-2]]))



In [ ]:
probs = np.array([
    [0.6, 0.4],   # t=0: blank=0.6, A=0.4
    [0.3, 0.7],   # t=1
    [0.2, 0.8]    # t=2
])

labels = [1]  # Target: A (blank=0 assumed at index 0)

print("Log CTC probability:", ctc_forward(probs, labels))


Log CTC probability: [2, 0]


In [ ]:
import numpy as np

# your log_probs example
log_probs = np.log(np.array([
    [0.6, 0.2, 0.1, 0.1], # t=0
    [0.1, 0.7, 0.1, 0.1], # t=1
    [0.1, 0.1, 0.7, 0.1], # t=2
    [0.1, 0.6, 0.1, 0.2], # t=3
]))
probs=np.exp(log_probs)

In [ ]:
def ctc_beam(probs,beam_size=2,blank=0):
  T,C=probs.shape
  beam={():1.0}

  for t in range(T):
    new_beam={}
    for seq,score in beam.items():
      for char in range(C):
        p=probs[t,char]

        if char==blank:
          new_seq=seq
        elif len(seq)>0 and seq[-1]==char:
          new_seq=seq
        else:
          new_seq=seq+(char,)

        new_score= score*p

        if new_seq in new_beam:
          new_beam[new_seq]+=new_score
        else:
          new_beam[new_seq]=new_score

    beam = dict(sorted(new_beam.items(), key=lambda x: x[1], reverse=True)[:beam_size])
  return max(beam,key=beam.get)


In [ ]:
decoded = ctc_beam(probs, beam_size=2, blank=3)
print(decoded)

(0, 1, 2, 1)


In [ ]:
def ctc_beam_1(probs):
  T,C=probs.shape

  beam={():1.0}

  for t in range(T):
    new_beam={}
    for seq,score in beam.items():
      for char in range(C):
        p=probs[t,char]

        if char==0:
          new_seq=seq
        elif len(seq)>0 and seq[-1]==char:
          new_seq=seq
        else:
          new_seq=seq+(char,)

        new_score=p*score

        if new_seq in new_beam:
          new_beam[new_seq]+=new_score
        else:
          new_beam[new_seq]=new_score
    beam=dict(sorted(new_beam.items(),key=lambda x:x[1],reverse=True)[:2])
  return max(beam,key=beam.get)


In [ ]:
decoded = ctc_beam_1(probs)
print(decoded)

(1, 2, 1)


In [ ]:
def greedy(log_probs):
  seq=np.argmax(log_probs,axis=1)

  out,prev=[],0

  for s in seq:
    if s!=0 and s!=prev:
      out.append(s)
    prev=s

  return out


In [ ]:
log_probs = np.log(np.array([
    [0.6, 0.2, 0.1, 0.1],
    [0.1, 0.7, 0.1, 0.1],
    [0.1, 0.1, 0.7, 0.1],
    [0.1, 0.6, 0.1, 0.2],
]))

decoded = greedy(log_probs)
print(decoded)


[np.int64(1), np.int64(2), np.int64(1)]
